## 自主进化 Agent 框架 illufly 的智能体论文实践：（1）打开魔法工具箱

前文我们粗略了解 illufly 基于对话获得经验的方法，并基于经验实现简单 RAG 的能力。

本文开始将探讨一系列的复杂推理场景和智能体论文实践，同时解读 illufly 如何在其中实践自主进化的理念。

一般来说，无论是哪一种 Agent 都需要使用工具。

因此，这个系列文章从介绍 illufly 的工具使用开始。

在 illufly 中，所有对话模型都是工具，所有 python 函数也都可以当作工具使用。<br>
另外，将来要介绍的条件选择器、流程智能体等也都可以当作工具使用。

### 直接将对话模型作为工具使用

In [10]:
from illufly.chat import ChatQwen

naming = ChatQwen(description="擅长给人起名字的专家")
naming.tool_desc

{'type': 'function',
 'function': {'name': 'ChatQwen.4922217888',
  'description': '擅长给人起名字的专家',
  'parameters': {'type': 'object',
   'properties': {'prompt': {'type': 'string', 'description': '详细描述用户问题'}},
   'required': ['prompt']}}}

**所述代码中已经定义了一个工具，并且使用了 openai 兼容的工具描述架构。**

调用工具，仅需将其传递给对话模型的 tools 参数，注意 tools 是一个列表，这意味着你可以同时使用很多工具。

In [11]:
qwen = ChatQwen(tools=[naming])
qwen("我姓薛，给我儿子起一个两个字名字，跟修仙小说里的大神修士那样霸气")



[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_4d07a7d0721e4295a82dd1", "type": "function", "function": {"name": "ChatQwen.4922217888", "arguments": "{\"prompt\": \"我姓薛，希望给我儿子起一个两个字的名字，要有修仙小说里大神修士那种霸气的感觉。\"}"}}]
[AGENT] ChatQwen.4922217888
好的，考虑到您希望名字能有修仙小说中大神修士的霸气感，这里为您提供几个建议：

1. 薛凌霄 (Xue Lingxiao)：凌霄意为直上云霄，有着超凡脱俗、高高在上的意味。
2. 薛破军 (Xue Pojun)：破军代表勇往直前，不畏强敌，有破而后立之意。
3. 薛天擎 (Xue Tianqing)：天擎意味着支撑天空，象征着力量与领导力。
4. 薛无极 (Xue Wuji)：无极在道家哲学中指没有极限，代表着无限可能和强大的力量。
5. 薛云中 (Xue Yunzhong)：云中给人以飘逸出尘之感，同时也有着高高在上、超然物外的意境。

这些名字都带有较强的个人魅力和独特的韵味，适合追求个性和力量感的家长选择。希望这些建议能够帮到您！

给您的儿子起名时，结合修仙小说中的元素，可以考虑以下几个名字：

1. **薛凌霄** (Xue Lingxiao)：凌霄意为直上云霄，有着超凡脱俗、高高在上的意味。
2. **薛破军** (Xue Pojun)：破军代表勇往直前，不畏强敌，有破而后立之意。
3. **薛天擎** (Xue Tianqing)：天擎意味着支撑天空，象征着力量与领导力。
4. **薛无极** (Xue Wuji)：无极在道家哲学中指没有极限，代表着无限可能和强大的力量。
5. **薛云中** (Xue Yunzhong)：云中给人以飘逸出尘之感，同时也有着高高在上、超然物外的意境。

每个名字都有其独特的含义，希望这些建议能够帮助您找到满意的名字！如果还有其他需求或想要更多的选项，欢迎继续交流。


'给您的儿子起名时，结合修仙小说中的元素，可以考虑以下几个名字：\n\n1. **薛凌霄** (Xue Lingxiao)：凌霄意为直上云霄，有着超凡脱俗、高高在上的意味。\n2. **薛破军** (Xue Pojun)：破军代表勇往直前，不畏强敌，有破而后立之意。\n3. **薛天擎** (Xue Tianqing)：天擎意味着支撑天空，象征着力量与领导力。\n4. **薛无极** (Xue Wuji)：无极在道家哲学中指没有极限，代表着无限可能和强大的力量。\n5. **薛云中** (Xue Yunzhong)：云中给人以飘逸出尘之感，同时也有着高高在上、超然物外的意境。\n\n每个名字都有其独特的含义，希望这些建议能够帮助您找到满意的名字！如果还有其他需求或想要更多的选项，欢迎继续交流。'

### 修改工具定义，让你“言出法随”

上面的输出太过随意，也许你已经厌烦了，需要更精细的控制。

使用 tool_params 参数可以进一步定义参数的说明；<br>
而 memory 则给工具一个系统提示语，明确告诉对话智能体他该怎么干活，不要自己想怎么说就怎么说。

In [8]:
from illufly.chat import ChatQwen

naming = ChatQwen(
    name="茅山道士",
    description="擅长给人起名字的专家",
    tool_params={
        "prompt": "起名字的需求细节：默认为返回2个推荐的名字。"
    },
    memory=[
        (
            "system",
            "你是起名专家。除了明确提出的需求，再用一句话讲出名字中包含的金、木、水、火、土符合风水学的依据，如果没有明确要求就给出两个推荐的名字"
        )]
)
naming.tool_desc

{'type': 'function',
 'function': {'name': '茅山道士',
  'description': '擅长给人起名字的专家',
  'parameters': {'type': 'object',
   'properties': {'prompt': {'type': 'string',
     'description': '起名字的需求细节：默认为返回2个推荐的名字。'}},
   'required': ['prompt']}}}

In [9]:
qwen = ChatQwen(tools=[naming])
qwen("我姓薛，给我儿子起一个两个字名字，跟修仙小说里的大神修士那样霸气")



[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_2b780aaf26f9417bb8e8a7", "type": "function", "function": {"name": "茅山道士", "arguments": "{\"prompt\": \"我姓薛，希望给我儿子起一个两个字的名字，感觉像修仙小说中的大神修士一样霸气。\"}"}}]
[AGENT] 茅山道士
可以考虑“薛霆”和“薛渊”。这两个名字都透着一股不凡的气息，仿佛修仙小说中的大神修士。其中，“霆”属金，象征雷霆万钧之力；“渊”属水，寓意深不可测的智慧与内敛的威严。

我为您想的两个名字是“薛霆”和“薛渊”。这两个名字都很有气势，就像修仙小说中的大神修士。“霆”象征雷霆万钧的力量，“渊”则意味着深不可测的智慧与内敛的威严。希望您会喜欢！


'我为您想的两个名字是“薛霆”和“薛渊”。这两个名字都很有气势，就像修仙小说中的大神修士。“霆”象征雷霆万钧的力量，“渊”则意味着深不可测的智慧与内敛的威严。希望您会喜欢！'

### 使用 python 函数自定义工具

有时候你需要结合业务情况，自己编写工具函数，直接使用 python 函数即可。

In [12]:
def tianqi(city: str):
    """查询天气可以找我"""
    return {"city": f"今天有雷雨"}

qwen = ChatQwen(tools=[tianqi])
qwen("我想去惠州爬山，今天可以吗？")



[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_059b7be639784fc9bdfeb3", "type": "function", "function": {"name": "tianqi", "arguments": "{\"city\": \"惠州\"}"}}]
[AGENT] tianqi
{'city': '今天有雷雨'}
今天惠州有雷雨，可能不太适合去爬山。为了安全和体验考虑，建议你改天再去。


'今天惠州有雷雨，可能不太适合去爬山。为了安全和体验考虑，建议你改天再去。'

我们修改一下工具的参数说明，要求大模型在调用时必须提供城市。<br>
此时，你必须使用 BaseAgent 类补充参数信息。

In [14]:
from illufly.types import BaseAgent

def tianqi(city: str):
    """查询天气可以找我"""
    return {"city": f"今天有雷雨"}
tool = BaseAgent(tianqi, tool_params={"city": "你必须告诉我具体城市，而不能是其他地址"})

qwen = ChatQwen(tools=[tool])
qwen("我想去爬罗浮山，今天可以吗？")

这取决于今天的天气状况以及你的身体状况。我可以帮你查询一下当地的天气。

[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_338e94e9ddb34c778ca2a0", "type": "function", "function": {"name": "tianqi", "arguments": "{\"city\": \"博罗\"}"}}]
[AGENT] tianqi
{'city': '今天有雷雨'}
今天博罗有雷雨，可能不太适合去爬罗浮山。为了安全和体验，建议你改天再去。如果一定要出行，请准备好雨具并注意安全。


'今天博罗有雷雨，可能不太适合去爬罗浮山。为了安全和体验，建议你改天再去。如果一定要出行，请准备好雨具并注意安全。'

In [2]:
还不错，

{'type': 'function',
 'function': {'name': 'mytool',
  'description': '我还没有工具描述',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string', 'description': '请告诉我你关注哪个城市的天气'}},
   'required': ['city']}}}

### 数据分析工具

下面只再看一个数据分析工具的例子。<br>
PandasAgent 是基于 pandas 库进行数据分析的 BaseAgent 子类，要求提供一个 pandas 数据框作为参数来构建 Dataset 实例。<br>

由于数据分析能力是 illulfy 的核心能力之一，后面还会从复杂工具规划、自主进化等角度反复探讨这一话题，这里仅做初步展示。

In [16]:
from illufly.chat import ChatQwen
from illufly.toolkits import PandasAgent
from illufly.types import Dataset
import pandas as pd

data = pd.DataFrame({
    "姓名": ["薛一凡", "肖一笑", "赖国良", "吴红兵"],
    "语文": [90, 80, 105, 110],
    "数学": [135, 110, 120, 90]
})

pan = PandasAgent(ChatQwen(), [Dataset(name="考试成绩", df=data)], name="分析师")
talker = ChatQwen(tools=[pan])

talker("根据成绩单，理科和文科最好的学生分别是谁？")



[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_e37f3a15183f44f39bead4", "type": "function", "function": {"name": "分析师", "arguments": "{\"question\": \"根据成绩单，找出理科和文科最好的学生。\"}"}}]
[AGENT] 分析师
为了完成这个任务，我们首先需要定义什么是“理科”和“文科”。在这个数据集中，数学可以被视为理科科目，而语文可以被视为文科科目。因此，我们可以分别找出数学和语文成绩最高的学生来确定理科和文科最好的学生。

下面是实现这一目标的Python代码：

```python
def main():
    # 获取数据集
    df = datasets['考试成绩'].df
    
    # 找出数学成绩最高的学生
    best_math_student = df.loc[df['数学'].idxmax()]
    
    # 找出语文成绩最高的学生
    best_chinese_student = df.loc[df['语文'].idxmax()]
    
    # 创建一个新的数据框来存储结果
    result_df = pd.DataFrame({
        '科目': ['数学', '语文'],
        '最高分': [best_math_student['数学'], best_chinese_student['语文']],
        '姓名': [best_math_student['姓名'], best_chinese_student['姓名']]
    })
    
    # 将结果数据框添加到数据集中
    add_dataset(result_df, "最佳学生", "理科和文科成绩最好的学生")
    
    return result_df
```

这段代码首先获取了“考试成绩”数据集，然后分别找到了数学和语文成绩最高的学生，并创建了一个新的数据框来存储这些信息。最后，将这个新的数据框添加到了数据集中，并返回了这个数据框。
   科目  最高分   姓名
0  数学  135  薛一凡
1  语文  110  吴红兵
根

'根据成绩单，理科（以数学成绩为代表）最好的学生是薛一凡，而文科（以语文成绩为代表）最好的学生是吴红兵。'

### 本文小结

前面探讨了在 illufly 中定义工具定义的方式。包括使用对话模型、数据分析和自定义工具等多种方式。

illufly 中集成了关于数据分析、写作、文生图、文生视频等很多 BaseAgent 子类，限于篇幅和本文作为初步引导的定位不展开叙述。

### 下期内容

illufly 内置实现了 ReAct、ReWoo、Plan and Solve 等流行的单智能体论文的实践。<br>

| FlowAgent子类 | 推理方式 | 论文来源 |
|:----|:--------|:------------|
|ReAct|一边推理一边执行|[ReAct](https://arxiv.org/abs/2210.03629) |
|ReWOO|一次性规划所有步骤后一起执行|[ReWOO](https://arxiv.org/abs/2305.18323) |
|PlanAndSolve|一边修订总体计划一边执行|[Plan-and-Solve](https://arxiv.org/abs/2305.04091) |

另外，illufly 也支持一些多智能体实现的策略。

下期开始首先讲述 illufly 中对 ReAct 推理的实践。